# PA 예측하기

## 1. Batter

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_probability as tfp

In [ ]:
batter = pd.read_csv("https://raw.githubusercontent.com/njj06135/Baseball_ChilliShrimp/master/data/modeling_batter_stadium.csv")

In [ ]:
batter

,year,month,T_ID,P_ID,PA,VS_T_ID_HH,VS_T_ID_HT,VS_T_ID_KT,VS_T_ID_LG,VS_T_ID_LT,VS_T_ID_NC,VS_T_ID_OB,VS_T_ID_SK,VS_T_ID_SS,VS_T_ID_WO,BAT_ORDER,SB_rate,BABIP,H2_rate,H3_rate,HR_rate,BB_rate,HP_rate,GD_rate,XR,AVG,OBP,SLG,OPS,stadium_MH,stadium_MS,stadium_JS,stadium_SW,stadium_DJ,stadium_DG,stadium_GJ,stadium_GC,stadium_SJ,stadium_PH,stadium_US,stadium_CJ,stadium_CW
0,2016,4,HH,60404,12,0,0,0,2,0,2,1,0,0,2,9,0.000000,0.142857,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.076,0.166667,0.166667,0.166667,0.333333,0,3,5,0,12,0,0,0,3,0,0,0,0
1,2016,4,HH,60667,10,0,0,0,2,3,0,3,0,0,0,8,0.000000,0.380952,0.333333,0.0,0.000000,0.100000,0.000000,0.100000,1.020,0.333333,0.400000,0.444444,0.844444,0,3,5,0,12,0,0,0,3,0,0,0,0
2,2016,4,HH,60757,0,0,0,0,0,0,0,1,0,0,0,9,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0,3,5,0,12,0,0,0,3,0,0,0,0
3,2016,4,HH,60805,1,0,0,0,0,0,0,0,0,2,0,2,0.000000,0.260870,0.000000,0.0,0.000000,1.000000,0.000000,0.000000,0.340,0.000000,1.000000,0.000000,0.000000,0,3,5,0,12,0,0,0,3,0,0,0,0
4,2016,4,HH,61700,36,0,1,0,3,3,3,3,0,0,3,8,0.533333,0.309133,0.166667,0.0,0.166667,0.111111,0.027778,0.083333,2.444,0.193548,0.305556,0.322581,0.628136,0,3,5,0,12,0,0,0,3,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7419,2020,9,SS,63435,4,3,3,4,2,2,3,4,2,0,2,0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,-0.368,0.000000,0.000000,0.000000,0.000000,0,0,4,2,0,13,1,0,2,0,0,0,2
7420,2020,9,SS,69418,0,3,3,4,2,2,3,4,2,0,2,7,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0,0,4,2,0,13,1,0,2,0,0,0,2
7421,2020,9,SS,61457,0,3,3,4,2,2,3,4,2,0,2,0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0,0,4,2,0,13,1,0,2,0,0,0,2
7422,2020,9,HH,66707,7,0,4,2,2,2,2,5,2,3,4,8,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,-0.662,0.000000,0.000000,0.000000,0.000000,0,0,1,2,16,2,2,2,0,0,0,0,0


In [ ]:
batter['G'] = batter['VS_T_ID_HH'] + batter['VS_T_ID_HT']+ batter['VS_T_ID_KT']+ batter['VS_T_ID_LG']+ batter['VS_T_ID_LT']+ batter['VS_T_ID_NC']+ batter['VS_T_ID_OB']+ batter['VS_T_ID_SK']+ batter['VS_T_ID_SS'] + batter['VS_T_ID_SS'] + batter['VS_T_ID_WO']
batter['PA_PER_G'] = batter['PA'] / batter['G']
batter = batter[['year', 'month', 'P_ID', 'PA_PER_G']]

In [ ]:
batter = batter.loc[batter.year == 2020,:]
batter_8 = batter.loc[batter.month > 7]

In [ ]:
batter_PID = batter_8.P_ID.unique()

In [ ]:
mean_pa = batter.groupby('P_ID').mean()
mean_pa = mean_pa[['PA_PER_G']]
std_pa = batter.groupby('P_ID').std()
std_pa = std_pa[['PA_PER_G']]

In [ ]:
sum_pa = batter.groupby('P_ID').sum()

In [ ]:
sum_pa = sum_pa[['PA_PER_G']]

In [ ]:
sum_pa['mean_pa'] = mean_pa
sum_pa['std_pa'] = std_pa

In [ ]:
sum_pa = sum_pa.fillna(0)
sum_pa = sum_pa[['mean_pa', 'std_pa']]

In [ ]:
sum_pa['P_ID'] = sum_pa.index

In [ ]:
predict_batter = pd.DataFrame()

for i in range(len(batter_PID)):
    temp = sum_pa.loc[sum_pa.P_ID == batter_PID[i],:]
    sampled = tfp.distributions.Normal(loc =temp.mean_pa, scale = temp.std_pa / 2).sample(1)
    sampled = sampled.numpy()
    sampled = np.append(batter_PID[i], sampled)
    sampled = pd.DataFrame(sampled.reshape(1,2))
    sampled.columns = ['P_ID', 'PA_PER_G']
    predict_batter = predict_batter.append(sampled)


In [ ]:
predict_batter.PA_PER_G[predict_batter.PA_PER_G < 0] = 0

In [ ]:
predict_batter

,P_ID,PA_PER_G
0,63202.0,0.188910
0,66209.0,0.133340
0,65462.0,0.086957
0,63260.0,0.874267
0,75151.0,1.981100
...,...,...
0,65057.0,0.000000
0,75340.0,0.000000
0,67449.0,0.200000
0,63435.0,0.248363


In [ ]:
predict_batter.to_csv("predicted_batter_pa.csv", index= False)

In [ ]:
from google.colab import files

In [ ]:
files.download("predicted_batter_pa.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## 2. Pitcher

In [ ]:
pitcher = pd.read_csv("https://raw.githubusercontent.com/njj06135/Baseball_ChilliShrimp/master/data/modeling_pitcher_stadium.csv")

In [ ]:
pitcher['G'] = pitcher['VS_T_ID_HH'] + pitcher['VS_T_ID_HT']+ pitcher['VS_T_ID_KT']+ pitcher['VS_T_ID_LG']+ pitcher['VS_T_ID_LT']+ pitcher['VS_T_ID_NC']+ pitcher['VS_T_ID_OB']+ pitcher['VS_T_ID_SK']+ pitcher['VS_T_ID_SS'] + pitcher['VS_T_ID_SS'] + pitcher['VS_T_ID_WO']
pitcher['PA_PER_G'] = pitcher['PA'] / pitcher['G']
pitcher = pitcher[['year', 'month', 'P_ID', 'PA_PER_G']]

In [ ]:
pitcher = pitcher.loc[pitcher.year == 2020,:]
pitcher_8 = pitcher.loc[pitcher.month > 7]
pitcher_PID = pitcher_8.P_ID.unique()

In [ ]:
mean_pa = pitcher.groupby('P_ID').mean()
mean_pa = mean_pa[['PA_PER_G']]
std_pa = pitcher.groupby('P_ID').std()
std_pa = std_pa[['PA_PER_G']]

In [ ]:
sum_pa = pitcher.groupby('P_ID').sum()
sum_pa = sum_pa[['PA_PER_G']]
sum_pa['mean_pa'] = mean_pa
sum_pa['std_pa'] = std_pa
sum_pa = sum_pa.fillna(0)
sum_pa = sum_pa[['mean_pa', 'std_pa']]
sum_pa['P_ID'] = sum_pa.index

In [ ]:
sum_pa

,mean_pa,std_pa,P_ID
P_ID,,,
50030,9.581667,8.745349,50030
50036,8.250000,1.767767,50036
50040,12.060810,8.844425,50040
50109,0.373219,0.310241,50109
50126,7.540000,8.148483,50126
...,...,...,...
79617,0.166667,0.026189,79617
79764,8.596713,10.946755,79764
79825,2.762731,2.509149,79825


In [ ]:
predict_pitcher = pd.DataFrame()

for i in range(len(sum_pa)):
    sampled = tfp.distributions.Normal(loc = sum_pa.iloc[i,:].mean_pa, scale = sum_pa.iloc[i,:].std_pa / 2).sample(1)
    sampled = sampled.numpy()
    sampled = np.append(int(sum_pa.iloc[i,:].P_ID), sampled)
    sampled = pd.DataFrame(sampled.reshape(1,2))
    sampled.columns = ['P_ID', 'PA_PER_G']
    predict_pitcher = predict_pitcher.append(sampled)

In [ ]:
predict_pitcher.PA_PER_G[predict_pitcher.PA_PER_G < 0] = 0

In [ ]:
predict_pitcher.to_csv("predicted_pitcher_pa.csv", index= False)

In [ ]:
from google.colab import files

In [ ]:
files.download("predicted_pitcher_pa.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>